In [1]:
# !pip install pandas
# !pip install sentence-transformers faiss-cpu
# !pip install --upgrade pip

# !pip install faiss-cpu
!pip install certifi

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import oracledb
from tabulate import tabulate
import random
import re
import webbrowser

# #SCRE
# username = 'svcGOCUI'
# password = 'DELL2023support#'
# host = 'gocplorlvpr18.amer.dell.com'
# port = '1521'
# service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'

#DAO GE4 SIT
dbname="DAO_GE4"
username = 'SIT_BROWSER'
password = 'browse_s1t'
host = 'udmnlorrrsi2a01.amer.dell.com'
port = '1521'
service_name = 'fdr4s.sit.amer.dell.com'
excel_name="DataFromDB_"+dbname+".xlsx"
def Retrive_VectorData():
    
    dsn = oracledb.makedsn(host, port, service_name=service_name)
    try :
        connection = oracledb.connect(user=username, password=password, dsn=dsn)   
        print("Connected to Oracle Database")
         
        cursor = connection.cursor()
        # query = '''SELECT t.owner || '.' || t.table_name AS Table_Name,c.column_name as Column_Name FROM all_tables t 
        # JOIN all_tab_columns c ON t.table_name = c.table_name  ORDER BY t.table_name, c.column_name'''

        query = '''SELECT t.owner || '.' || t.table_name AS Table_Name,t.column_name as Column_Name 
                from all_tab_columns t
                ORDER BY t.table_name, t.column_name'''

        cursor.execute(query)

        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        data = pd.DataFrame(rows,columns=column_names)
        print(excel_name)
       
        data.to_excel(excel_name,index=False)
        table = tabulate(rows, headers=column_names, tablefmt="pretty")
        print(data)
        # data = pd.read_excel("DataFromDB.xlsx")
       
       
        # print(type(data))
        # print(data)
        return data
     
    except Exception as e:
       print(f"An error occurred: {e}")
    finally:
        if connection:
            connection.close()
            print("Connection closed")

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

df_schema = Retrive_VectorData()

df_schema.columns = df_schema.columns.str.strip()
df_schema.columns = df_schema.columns.str.lower()
df_schema = df_schema.rename(columns={"table_name": "Table_Name", "column_name": "Column_Name"})
# print(df_schema)

# creating schema info
df_schema['schema_info'] = df_schema.apply(lambda row: f"{row['Table_Name']} ({row['Column_Name']})", axis=1)
print(df_schema['schema_info'])
# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

#Generating Vector data for schema
schema_embeddings = model.encode(df_schema['schema_info'].tolist(),show_progress_bar=True)

# Create FAISS index
d = schema_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(schema_embeddings)

C:\Users\BharathKumar_Bellam\AppData\Roaming\Python\Python310\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Connected to Oracle Database
DataFromDB_DAO_GE4.xlsx
                        TABLE_NAME           COLUMN_NAME
0       FSL_GOLF.AAS_MODULE_CONFIG  AUTHENTICATION_CLASS
1       AMC_3PLA.AAS_MODULE_CONFIG  AUTHENTICATION_CLASS
2       UFD_GOLF.AAS_MODULE_CONFIG  AUTHENTICATION_CLASS
3       UFD_GOLF.AAS_MODULE_CONFIG   AUTHORIZATION_CLASS
4       AMC_3PLA.AAS_MODULE_CONFIG   AUTHORIZATION_CLASS
...                            ...                   ...
179415     SYS._utl$_lnc_ind_parts                STATUS
179416     SYS._utl$_lnc_ind_parts              TABLE_ID
179417     SYS._utl$_lnc_ind_parts          TAB_OWNER_ID
179418     SYS._utl$_lnc_ind_parts             TCCOMP_ID
179419     SYS._utl$_lnc_ind_parts              TCOMP_ID

[179420 rows x 2 columns]
Connection closed
0         FSL_GOLF.AAS_MODULE_CONFIG (AUTHENTICATION_CLASS)
1         AMC_3PLA.AAS_MODULE_CONFIG (AUTHENTICATION_CLASS)
2         UFD_GOLF.AAS_MODULE_CONFIG (AUTHENTICATION_CLASS)
3          UFD_GOLF.AAS_MODULE_CONFIG 

Batches:   0%|          | 0/5607 [00:00<?, ?it/s]

In [4]:
df_schema.to_excel(excel_name,index=False)

In [5]:
def retrieve_schema_info(query, index, model, df_schema, top_k=15):
    query_embedding = model.encode([query])
    print('query_embedding',query_embedding)
    distances, indices = index.search(query_embedding, top_k)
    r = df_schema.iloc[indices[0]]['schema_info'].tolist()
    print('retrieve_embeddings',r)
    return r

# Example usage
user_query = "write a query to get order date for the sales order id 740151575"
relevant_schema_info = retrieve_schema_info(user_query, index, model, df_schema)
print(relevant_schema_info)

query_embedding [[-5.45608588e-02  1.41842932e-01  7.84974992e-02 -3.47444927e-03
  -1.46887749e-01  5.06235436e-02  6.71212096e-03  3.27527374e-02
   3.18324678e-02 -2.80218590e-02  1.79476931e-03  1.72822233e-02
   2.32173391e-02 -1.02501646e-01 -7.26658404e-02  2.66407207e-02
   2.28309352e-02  1.73426941e-02 -1.20357731e-02 -2.13319249e-02
   6.12435490e-03  1.12980291e-01 -8.84497911e-02 -8.83886404e-03
   1.08969780e-02 -7.18176039e-03  1.80418044e-02 -2.23290212e-02
  -3.12070027e-02  2.89423013e-04 -6.22797087e-02  2.75276527e-02
  -1.50560793e-02  3.42136025e-02  4.05442119e-02 -1.24690421e-01
   2.67482288e-02 -3.04599684e-02  6.12262264e-02 -4.03440781e-02
  -4.50024568e-02 -2.86367312e-02  7.73679018e-02  1.14784934e-01
  -3.76137979e-02 -7.24682733e-02  1.29703032e-02  7.30881914e-02
   5.77294789e-02  2.90597007e-02 -1.08807378e-01  1.42293144e-03
   3.64111178e-02 -1.51421186e-02  5.45263067e-02  3.12306322e-02
  -2.34772936e-02 -4.94558737e-02  1.86303374e-03  3.8667265

In [6]:
import json
import requests
import os
import warnings
import httpx
import warnings
few_shot_examples=[]
# Suppress only the single InsecureRequestWarning from urllib3 needed
warnings.filterwarnings('ignore', 'Unverified HTTPS request')

# As mentioned in the above Note try to setup Dell certificate in your environment to avoid verify=False(SSL verification is disabled).
# http_client=httpx.Client(verify=False)
streaming = True
max_output_tokens = 200

# Available Models list
available_model_apis = ["mpt-30b-instruct", "llama-2-13b-chat", "zephyr-7b-beta"]
# Let's select the model from available list
model_selected = available_model_apis[2]
api_key = 'ZTMxOWNhN2EtY2JmMS00ODcyLTg5ZjgtNDM0NGI3ZTY5ZmRi'

def stream_and_yield_response(response):
    for chunk in response.iter_lines():
        decoded_chunk = chunk.decode("utf-8")
        if decoded_chunk == "data: [DONE]":
            pass
        elif decoded_chunk.startswith("data: {"):
            payload = decoded_chunk.lstrip("data:")
            json_payload = json.loads(payload)
            yield json_payload['choices'][0]['text']

def llm_api(data):
    """
    Creates a request to Open Source GenAI Text to Text model with API key in header.
    """
     
    url = f"https://opensource-llm-api.aiaccel.dell.com/v1/completions"
      
    headers = {
        'accept': 'application/json',
        'api-key': api_key,
        'Content-Type': 'application/json'
    }
      
    try:
        response = requests.post(url, headers=headers, json=data, stream=data['stream'], verify=False)
        response.raise_for_status()
        res =""
        if data['stream']:
            # print(response.text)
            for result in stream_and_yield_response(response):
                print(result, end='')
                res+=result
            
        else:
            response_dict = response.json()
            result = response_dict['choices'][0]['text']
            # print(result)
            return result
        return res
           
    except requests.exceptions.HTTPError as err:
        print('Error code:', err.response.status_code)
        print('Error message:', err.response.text)
    except Exception as err:
        print('Error:', err)



def generate_sql_query_fewshotprompting(user_query, relevant_schema_info, llm_api_func):

    # Few-shot examples for CIS
    few_shot_examples = [
        {
            "prompt": "write a query to get MESSAGE_TYPE for the BUSINESS_LOOKUP_KEY = 870599090",
            "sql_command": "SELECT MESSAGE_TYPE FROM transaction_log WHERE BUSINESS_LOOKUP_KEY = '870599090'"
        },
        {
            "prompt": "write a query to get SHIPMENT_NUMBER for the sales_order_id = 852424981",
            "sql_command": "SELECT SHIPMENT_NUMBER FROM MWMGR.SALES_ORDER_DETAIL WHERE SALES_ORDER_ID = '852424981'"
        },
        {
            "prompt": "write a query to get INBOUND_TIME for the BUSINESS_LOOKUP_KEY = 399845462",
            "sql_command": "SELECT INBOUND_TIME FROM MWMGR.SERVICE_ORDER_LOGS WHERE BUSINESS_LOOKUP_KEY = '399845462'"
        }
    ]
    
   # Create the context with few-shot examples
    context = "Below are some examples of SQL queries:\n"
    for example in few_shot_examples:
        context += f"prompt: {example['prompt']}\nSQL_command: {example['sql_command']}\n\n"

    # Add the relevant schema information and user query
    context += f"Relevant information: {', '.join(relevant_schema_info)}\n"

    # Call the LLM API to generate the query
    instruction_text = f"Can you output a single SQL Query for the given prompt:{user_query}, based on the following context: {context} "
    # print(instruction_text)
    data = {
        'prompt': f'{instruction_text}',
        'temperature': 0.5,
        'top_p': 0.95,
        'max_tokens': 200,
        'stream': True,
        'model': model_selected
    }
    
    print("LLM Generated data")
    
    generated_query = llm_api_func(data)
    data = generated_query.split("Query")[0]
    response = data.strip()
    
    return response

LLM_Response = generate_sql_query_fewshotprompting(user_query, relevant_schema_info, llm_api)
print(LLM_Response)


LLM Generated data
Error code: 503
Error message: {'status_code': 503, 'detail': "b'<html>\\r\\n<head><title>503 Service Temporarily Unavailable</title></head>\\r\\n<body>\\r\\n<center><h1>503 Service Temporarily Unavailable</h1></center>\\r\\n<hr><center>nginx</center>\\r\\n</body>\\r\\n</html>\\r\\n'"}


AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
import cx_Oracle
from transformers import pipeline
import httpx
# #SCRE Oracle database connection details
# username = 'svcGOCUI'
# password = 'DELL2023support#'
# host = 'gocplorlvpr18.amer.dell.com'
# port = '1521'
# service_name = 'gooap_rw_oud_tls.prd.amer.dell.com'

#DAO GE4 SIT
username = 'SIT_BROWSER'
password = 'browse_s1t'
host = 'udmnlorrrsi2a01.amer.dell.com'
port = '1521'
service_name = 'fdr4s.sit.amer.dell.com'
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)

def connect_to_oracle():
    try:
        connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
        print("Connected to Oracle Database")
        return connection
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Oracle error code: {error.code}")
        print(f"Oracle error message: {error.message}")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def execute_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

        if rows:
            print("rows:",rows)
            table = tabulate(rows, headers=column_names, tablefmt="pretty")
            return table
        else:
            print("No results found.")
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Oracle error code: {error.code}")
        print(f"Oracle error message: {error.message}")
        return error.code
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()

def is_sql_query(user_input):
    sql_keywords = ["SELECT", "INSERT", "UPDATE", "DELETE", "FROM", "WHERE", "JOIN", "CREATE", "DROP", "ALTER"]
    return any(keyword in user_input.upper() for keyword in sql_keywords)

def main():
    # print(few_shot_examples)
    connection = connect_to_oracle()
    if not connection:
        return
    
    # Load the transformer model
    # model="bert-large-uncased-whole-word-masking-finetuned-squad"
    nlp = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

    try:
        while True:
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                break

            if is_sql_query(user_input):
                execute_query(connection, user_input)
            else:
                relevant_schema_info = retrieve_schema_info(user_input, index, model, df_schema)
                sql_query = generate_sql_query_fewshotprompting(user_input, relevant_schema_info, llm_api)
                # generate_sql_query(user_query, relevant_schema_info, llm_api)
                print("sql query",sql_query)
                response_from_db = execute_query(connection, sql_query)
                if response_from_db == 933:
                    continue
                print("response from db",response_from_db)
                print("Does the response relevent to your question yes/no")
                
                input_response = input("You: ").strip()
                if input_response.lower() == 'yes':
                    res={
                        "query": user_input,
                        "sql": response_from_db
                    }
                    few_shot_examples.append(res)
                # print(few_shot_examples)
                    
    finally:
        if connection:
            connection.close()
            print("Connection closed")

if __name__ == "__main__":
    main()
